In [23]:
from twilio.rest import TwilioRestClient

In [24]:
def load_twilio_config():
    twilio_account_sid = os.environ.get('TWILIO_ACCOUNT_SID')
    twilio_auth_token = os.environ.get('TWILIO_AUTH_TOKEN')
    twilio_number = os.environ.get('TWILIO_NUMBER')
    
    if not all([twilio_account_sid, twilio_auth_token, twilio_number]):
        logger.error(NOT_CONFIGURED_MESSAGE)
        raise MiddlewareNotUsed

    return (twilio_number, twilio_account_sid, twilio_auth_token)

In [25]:
class TwilioNotificationsMiddleware(object):
    def __init__(self):
        #self.administrators = load_admins_file()
        self.administrators = [dict(name="Joe", phone_number="+num")]
        self.client = MessageClient()

    def process_exception(self, request, exception):
        exception_message = str(exception)
        message_to_send = MESSAGE % exception_message

        for admin in self.administrators:
            self.client.send_message(message_to_send, admin['phone_number'])

        logger.info('Administrators notified')

        return None

In [26]:
class MessageClient(object):
    def __init__(self):
        (twilio_number, twilio_account_sid,
         twilio_auth_token) = load_twilio_config()

        self.twilio_number = twilio_number
        self.twilio_client = TwilioRestClient(twilio_account_sid,
                                              twilio_auth_token)

    def send_message(self, body, to):
        self.twilio_client.messages.create(body=body, to=to,
                                           from_=self.twilio_number,
                                           # media_url=['https://demo.twilio.com/owl.png'])
                                           )
    
    def make_call(self, to):
        self.twilio_client.calls.create(to=to,
                                           from_=self.twilio_number,
                                           url="http://twimlets.com/holdmusic?Bucket=com.twilio.music.ambient"
                                           )

In [27]:
m = TwilioNotificationsMiddleware()

NameError: name 'os' is not defined

In [28]:
numlist = [("dan", "+num1"), ("allen", "+num2")]

In [29]:
for name, num in numlist:
    print("sending message to {}".format(name))
    m.client.send_message("hi {}".format(name), num)

sending message to dan


TwilioRestException: HTTP 400 error: The 'To' number +num1 is not a valid phone number.

In [30]:
m.client.make_call(to="+num")

TwilioRestException: HTTP 400 error: The phone number you are attempting to call, 686, is not valid.